In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import random

import plotly.express as px
import chart_studio.plotly as py
import chart_studio.tools as tls
import os

In [2]:
%env PLOTLY_USER=arhanna
%env PLOTLY_API=6EStzo8mO5ZY6kfaehPe

env: PLOTLY_USER=arhanna
env: PLOTLY_API=6EStzo8mO5ZY6kfaehPe


In [3]:
user = %env PLOTLY_USER
api = %env PLOTLY_API

tls.set_credentials_file(username= user, api_key= api)

# Data Loading Stage + Pre-processing

In [4]:
# Custom dataset
# Added custom column header line to the csv after download
df = pd.read_csv('./data.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df.describe()

,id,confidence
count,1.487000e+03,1487.000000
mean,1.358533e+18,0.729403
std,1.806123e+16,0.133144
min,1.240376e+18,0.500375
25%,1.359173e+18,0.611370
50%,1.364312e+18,0.726221
75%,1.365790e+18,0.836797
max,1.366625e+18,0.996064


In [5]:
df.head()

,text,user,id,created_at,confidence,sentiment
0,@jeff_foust Rockets are ridiculously hard (sigh),elonmusk,1364707463510040586,2021-02-24 22:43:08+00:00,0.860945,Negative
1,@VitoTheGreat_ I love dogs &amp; memes,elonmusk,1364564687396872193,2021-02-24 13:15:48+00:00,0.730632,Positive
2,@konstructivizm Hey babe 😉,elonmusk,1365122612217712642,2021-02-26 02:12:48+00:00,0.860025,Positive
3,On the actual moon,elonmusk,1364563280522453005,2021-02-24 13:10:13+00:00,0.770221,Positive
4,Literally https://t.co/XBAUqiVsPH,elonmusk,1364560733472579591,2021-02-24 13:00:05+00:00,0.552344,Negative


### Setup groups/counts/sums

In [6]:
# df_s: change sentiment to ints: 1,-1
df_s = df.copy()
df_s['sentiment'] = df_s['sentiment'].apply(lambda x: 1 if x=="Positive" else -1)
df_s.head()

,text,user,id,created_at,confidence,sentiment
0,@jeff_foust Rockets are ridiculously hard (sigh),elonmusk,1364707463510040586,2021-02-24 22:43:08+00:00,0.860945,-1
1,@VitoTheGreat_ I love dogs &amp; memes,elonmusk,1364564687396872193,2021-02-24 13:15:48+00:00,0.730632,1
2,@konstructivizm Hey babe 😉,elonmusk,1365122612217712642,2021-02-26 02:12:48+00:00,0.860025,1
3,On the actual moon,elonmusk,1364563280522453005,2021-02-24 13:10:13+00:00,0.770221,1
4,Literally https://t.co/XBAUqiVsPH,elonmusk,1364560733472579591,2021-02-24 13:00:05+00:00,0.552344,-1


In [7]:
df_g = df_s.copy()
df_g_sentiment = df_s.groupby(['user', pd.Grouper(key='created_at', freq='D')])['sentiment'].sum().reset_index(name ='sentiment')
df_g.head()
# df_g.describe()

,text,user,id,created_at,confidence,sentiment
0,@jeff_foust Rockets are ridiculously hard (sigh),elonmusk,1364707463510040586,2021-02-24 22:43:08+00:00,0.860945,-1
1,@VitoTheGreat_ I love dogs &amp; memes,elonmusk,1364564687396872193,2021-02-24 13:15:48+00:00,0.730632,1
2,@konstructivizm Hey babe 😉,elonmusk,1365122612217712642,2021-02-26 02:12:48+00:00,0.860025,1
3,On the actual moon,elonmusk,1364563280522453005,2021-02-24 13:10:13+00:00,0.770221,1
4,Literally https://t.co/XBAUqiVsPH,elonmusk,1364560733472579591,2021-02-24 13:00:05+00:00,0.552344,-1


In [8]:
df_g_sentiment.head()

,user,created_at,sentiment
0,AndrewYang,2021-02-21 00:00:00+00:00,0
1,AndrewYang,2021-02-22 00:00:00+00:00,8
2,AndrewYang,2021-02-23 00:00:00+00:00,11
3,AndrewYang,2021-02-24 00:00:00+00:00,10
4,AndrewYang,2021-02-25 00:00:00+00:00,7


In [9]:
# fig, ax = plt.subplots(figsize=(30,10))
cdm = {"Positive": "Blue", "Negative": "Red"}
co ={"sentiment": ["Positive", "Negative"]}
df_user_sum = df.groupby(['user', 'sentiment'])['sentiment'].count().reset_index(name='count')
fig = px.bar(df_user_sum, x="user", y="count",color="sentiment", title="User Total", category_orders=co)
url = py.plot(fig, filename='user_tweets')
df_user_sum.head()

,user,sentiment,count
0,AndrewYang,Negative,26
1,AndrewYang,Positive,74
2,CNN,Negative,51
3,CNN,Positive,124
4,DonaldJTrumpJr,Negative,22


### Specific User analysis

In [10]:
user = random.choice(df.user.unique())
df_u = df_s[df_s['user']==user].groupby(['user', pd.Grouper(key='created_at', freq='H')])['sentiment'].sum().reset_index(name='sum')
df_u['sentiment'] = df_u['sum'].apply(lambda x: "Negative" if x < 0 else "Positive")
len(df_u['sentiment'])

11

In [11]:
fig = px.bar(df_u, x="created_at", y="sum", color="sentiment", category_orders=co, title=f'{user}: Tweets',hover_data=df_u.columns)
url = py.plot(fig, filename=f'{user}_bar', auto_open=False)

In [12]:
print(url)

https://plotly.com/~arhanna/40/
